In [1]:
import pandas as pd
import os
import glob
import anndata as ad
from scipy.sparse import csr_matrix
import numpy as np
from ome_zarr.io import parse_url
import zarr
from ngff_tables_prototype.writer import write_table_points

/home/kyoda/ngff/lib/python3.10/site-packages/anndata/experimental/pytorch/_annloader.py:18: UserWarning: Сould not load pytorch.
  warnings.warn("Сould not load pytorch.")


In [2]:
df = pd.read_csv('wt-N2-081015-01.csv', names=('t', 'pid', 'cid', 'x', 'y', 'z', 'name', 'sphericity', 'volume', 'phase'))
df['entity'] = 'line'


print(df.head())

print(df)

   t   pid   cid        x        y        z name  sphericity    volume phase  \
0  1     0  1000  31.2026  11.0956  12.5788   P0    0.837884   76238.3     N   
1  2  1000  2000  31.4154  11.5594  12.7917   P0    0.811657   88108.6     N   
2  3  2000  3000  32.0421  11.3223  12.5404   P0    0.854491  110026.0     N   
3  4  3000  4000  32.5862  11.4526  12.4790   P0    0.771704  107802.0     N   
4  5  4000  5000  33.7902  11.7388  11.4331   P0    0.815484  138333.0     N   

  entity  
0   line  
1   line  
2   line  
3   line  
4   line  
     t    pid    cid        x        y         z name  sphericity    volume  \
0    1      0   1000  31.2026  11.0956  12.57880   P0    0.837884   76238.3   
1    2   1000   2000  31.4154  11.5594  12.79170   P0    0.811657   88108.6   
2    3   2000   3000  32.0421  11.3223  12.54040   P0    0.854491  110026.0   
3    4   3000   4000  32.5862  11.4526  12.47900   P0    0.771704  107802.0   
4    5   4000   5000  33.7902  11.7388  11.43310   P0    0

In [3]:
n_obs, n_vars = 65, 4
nmp = df[["t", "z", "y", "x"]].values
Y = nmp
X1 = Y / np.array([1, 0.5016, 0.1015, 0.1015])
X2 = X1 * np.array([1, -1, 1, 1])
X = X2 + np.array([0, 65, 0, 0])

dfa = pd.DataFrame(X, columns=list('tzyx'), index=np.arange(n_obs, dtype=int).astype(str))

In [4]:
obs_raw = df.loc[:,["cid", "entity", "name", "sphericity", "volume"]]

obs_meta = obs_raw.rename(columns={'cid': 'id'})

col = obs_meta.loc[:, 'id']
col.transpose()

# NB: use int8 to allow viewing in web
obsp_raw = np.loadtxt('wt-N2-081015-01-track.csv', delimiter=',', dtype=np.int8)
obsp_meta = pd.DataFrame(obsp_raw, index = col.astype(str), columns = col.astype(str))


In [5]:
print(obsp_meta)

id     1000  2000  3000  4000  5000  6000  7000  8000  9000  10000  ...  \
id                                                                  ...   
1000      0     1     0     0     0     0     0     0     0      0  ...   
2000      0     0     1     0     0     0     0     0     0      0  ...   
3000      0     0     0     1     0     0     0     0     0      0  ...   
4000      0     0     0     0     1     0     0     0     0      0  ...   
5000      0     0     0     0     0     1     0     0     0      0  ...   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...    ...  ...   
48001     0     0     0     0     0     0     0     0     0      0  ...   
49000     0     0     0     0     0     0     0     0     0      0  ...   
49001     0     0     0     0     0     0     0     0     0      0  ...   
50000     0     0     0     0     0     0     0     0     0      0  ...   
50001     0     0     0     0     0     0     0     0     0      0  ...   

id     46000  46001  470

In [6]:
adata = ad.AnnData(X = dfa, obs = obs_meta)

/tmp/ipykernel_10974/2054089064.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(X = dfa, obs = obs_meta)
/home/kyoda/ngff/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [7]:
# write as sparse data to obsp
adata.obsp["tracking"] = csr_matrix(obsp_raw)

In [8]:
store = parse_url("wt-N2-081015-01.ome.zarr", mode="w").store
root = zarr.group(store=store)

tables_group = root.create_group(name="tables")
write_table_points(
    group=tables_group,
    adata=adata
)

/home/kyoda/ngff/lib/python3.10/site-packages/zarr/creation.py:250: UserWarning: ignoring keyword argument 'maxshape'
  warn('ignoring keyword argument %r' % k)
